# Implementation of a 3D Unet Architecture


In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os, time
from importlib import reload

# 3D visualization tools
from mayavi import mlab
mlab.init_notebook(backend='ipy')

import tensorflow as tf
import model, utilities

Notebook initialized with ipy backend.


In [2]:
# Import modules providing tools for image manipulation
import sys
sys.path.append('../tools/')
import mosaic, deformation, affine 

In [3]:
# Reload of custom libraries to test hot-fixes
reload(utilities)
reload(model)
reload(deformation)
reload(affine)

<module 'affine' from '../tools\\affine.py'>

In [4]:
# Fix for tensorflow-gpu issues that I found online... (don't ask me what it does)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Convolutional Network Arithmetics

### Graphics memory
While the architecture of the Unet is agnostic regarding the shape of it's 3D Input efficient evaluation demands constant input size.
The first call of the Unet model triggers the allocation of GPU memory. Beside the massive number of parameters, the 3D tensors itself have large memory requirements. The maximum input tensor size is therefore directly determined by the available graphics memory of the GPU.

The size of 220px^3 input cubes with 8 initial filters has been determined to lie close to the memory max of my personal computer (6 GB graphics memory)
On the Janelia desktop computer (12GB graphics memory) up to 300px^3 cubes can be used.

The voxel size of the single fru labeled neuron dataset is specified as roghly (0.1 um x 0.1 um x 0.18 um).
### Reduction of output size
Since the Unet architecture relies only on valid convolution operations, the size of the output tensor is reduced with respect to the input. The unet architecture requires an even tensor shape before every max pooling operation. This narrows down the permissible input sizes. utilities.check_size() performs the necessary calculations to check which input sizes are valid and what output dimensions result from them.

In [5]:
# Get permissible network input sized (cube lengths)
n_blocks = 2 # The number of downsample und upsample blocks in each branch of the network.
valid_inputs = [n for n in range(512) if utilities.check_size(n, n_blocks=n_blocks)[0]]
print(valid_inputs)
# Check ouput for a given input size
cube_length = 220
output_length = utilities.check_size(cube_length, n_blocks)[1]
print('Output shape at {} is {} (mask_crop = {})'.format(cube_length,output_length,
                                                        (cube_length-output_length)/2))

[92, 100, 108, 116, 124, 132, 140, 148, 156, 164, 172, 180, 188, 196, 204, 212, 220, 228, 236, 244, 252, 260, 268, 276, 284, 292, 300, 308, 316, 324, 332, 340, 348, 356, 364, 372, 380, 388, 396, 404, 412, 420, 428, 436, 444, 452, 460, 468, 476, 484, 492, 500, 508]
Output shape at 220 is 132.0 (mask_crop = 44.0)


## Data Input Pipeline

The whole dataset has a size in the order of terabytes. At this stage we only work with preextracted volumes of (220px)^3 size.
Heavy use of data augumentation should enable the network to learn efficiently from a very low number of annotated samples.

the utilities module offers a custom keras.util.Sequence object that performs real time data augumentation. At the moment these operations consume a lot of time and could take up a lot of additional time


In [6]:
# Load some slices from the dataset

# Locate the sample directory on the computer
base_dir = 'C:\\Users\\Linus Meienberg\\Documents\\ML Datasets\\FruSingleNeuron_20190707\\SampleCrops'
samples = os.listdir(base_dir)

# Load all sample training data
train_samples = [utilities.load_volume(os.path.join(base_dir, sample)) for sample in samples[:2]] # List of dicts
train_images = [sample['image'] for sample in train_samples] # List of image tensors
train_masks = [sample['mask'] for sample in train_samples] # List of mask tensors

# Load all sample test data
test_samples = [utilities.load_volume(os.path.join(base_dir, sample)) for sample in samples[2:]] # List of dicts
test_images = [sample['image'] for sample in test_samples] # List of image tensors
test_masks = [sample['mask'] for sample in test_samples] # List of mask tensors

In [7]:
#TODO explore techniques to speed up image augumentation and batch preparation
train_sequence = utilities.Dataset3D(batch_size=1, batches = 1, images=train_images, masks=train_masks, mask_crop=44, augument=True, elastic=False, affine=True)
test_sequence  = utilities.Dataset3D(batch_size=1, batches= 1, images=test_images, masks=test_masks, mask_crop=44, augument= False)

## Visualization and augumentation tools

The following section is a quick demonstration of the 3D visualization and image augumentation tools provided by the utilities, affine and deformation module.

In [31]:
#Load the first sample as a test case
sample = utilities.load_volume(os.path.join(base_dir, samples[0])) # Dicttionary holding image and mask tensor
utilities.show3DImage(sample['image']) # Visualize image data using some isosurfaces

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [54]:
utilities.show3DImage(sample['mask'], mode='mask') # Visualize mask data with slightly different parameters

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [33]:
np.histogram(sample['mask'], bins=3)
#BUG The mask seems to contain integer values up to 3 how does that come? what does it signify? 

(array([28756806,  3184260,     2934], dtype=int64),
 array([0., 1., 2., 3.], dtype=float32))

In [71]:
# Illustrate the use of the elastic deformation tool implemented in the deformation module
displacement = deformation.displacementGridField3D((220,220,220), n_lines=5) # define a 3D vector field that is applied to the image coordinates
sample_img_deformed = deformation.applyDisplacementField3D(sample['image'], *displacement)
sample_mask_deformed = deformation.applyDisplacementField3D(sample['mask'], *displacement, interpolation_order= 0)

In [72]:
utilities.show3DImage(sample_img_deformed)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [12]:
# Directly visualize a tensor in the batch structure generated by utilities.Dataset3D
test_batch = train_sequence.__getitem__(0)
utilities.show3DImage(test_batch[0][0,...])

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xf4\x00\x00\x01\xf4\x08\x02\x00\x00\x00D\xb4H\xd…

In [37]:
pred.shape

TensorShape([1, 132, 132, 132, 2])

## Training the 3D Unet


In [14]:
reload(model)

<module 'model' from 'c:\\Users\\Linus Meienberg\\Google Drive\\Janelia\\ImageSegmentation\\3D Unet\\model.py'>

In [5]:
# Define the model (build is triggered on first call)
#unet = model.Unet(n_blocks=2, initial_filters=8)(inputs=tf.keras.layers.Input(shape=(220,220,220,1)))
unet = model.build_unet(input_shape=(220,220,220,1), n_blocks=2, initial_filters=2)
#test_batch = train_sequence.__getitem__(0)
#test_output = unet(test_batch[0])
#test_output.shape

crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]


In [6]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 220, 220, 22 0                                            
__________________________________________________________________________________________________
input_block (InputBlock)        ((None, 108, 108, 10 276         input_1[0][0]                    
__________________________________________________________________________________________________
downsample_block (DownsampleBlo ((None, 52, 52, 52,  1308        input_block[0][0]                
__________________________________________________________________________________________________
downsample_block_1 (DownsampleB ((None, 24, 24, 24,  5208        downsample_block[0][0]           
______________________________________________________________________________________________

In [17]:
unet.layers[1].get_config()

In [10]:
tf.keras.models.save_model(unet, 'test_save')

crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: test_save\assets


AttributeError: 'NoneType' object has no attribute 'items'

In [10]:
unet.compile(
    optimizer = tf.keras.optimizers.SGD(),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    run_eagerly = True
             )

In [13]:
print(unet(test_batch[0]).shape)
print(test_batch[1].shape)
unet.train_on_batch(x=test_batch[0], y=test_batch[1])

crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
(1, 132, 132, 132, 2)
(1, 132, 132, 132, 1)
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]


ResourceExhaustedError: OOM when allocating tensor with shape[1,48,136,136,136] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv3DBackpropInputV2]

In [13]:
history = unet.fit(train_sequence, epochs=25,
                   validation_data= test_sequence,
                   callbacks=[tf.keras.callbacks.ModelCheckpoint('3d_unet_0713.h5', save_best_only=True)]
                       )
                       

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1 steps, validate for 1 steps
Epoch 1/25
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]
crop = [4, 4, 4]
crop = [16, 16, 16]
crop = [40, 40, 40]


NotImplementedError: 

2